# EDA y Preparación — Dataset de 2 Clases (Natalie Portman vs Scarlett Johansson)

**Condición del curso:** Prohibido usar `scikit-learn`. Se utilizan **PyTorch, Torchvision, PIL, OpenCV, ImageHash, NumPy y Pandas**.

Este cuaderno realiza:
- Exploración general del dataset (conteo por clase, resoluciones, formatos, imágenes corruptas/vacías, balance de clases).
- Análisis estadístico de color por canal RGB (media, desviación), rangos globales y detección de imágenes atípicas (muy oscuras, borrosas).
- Preprocesamiento básico (redimensionado, normalización, conversión a tensores) con **Torchvision**.
- Definición de **data augmentation** para robustecer el modelo (rotaciones, flips, cambios de color, etc.).
- Detección de **duplicados** mediante *perceptual hash*.
- División **estratificada** en train/val sin usar sklearn.
- Exportación de CSVs para control de calidad manual.

### Estructura esperada de carpetas

```
data/
  raw/
    Natalie_Portman/
    Scarlett_Johansson/
```

Si tus carpetas tienen otros nombres, ajusta el diccionario `CLASS_MAP` más abajo.


In [ ]:
# Instalación de dependencias (solo se ejecuta automáticamente en Colab)
import sys
if 'google.colab' in sys.modules:
    !pip -q install opencv-python imagehash tqdm matplotlib pillow pandas numpy torch torchvision --upgrade

import os
from pathlib import Path
import numpy as np
import pandas as pd
from PIL import Image
import imagehash
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
from torchvision import transforms

print('✅ Librerías cargadas (sin scikit-learn)')

In [ ]:
# Configuración de rutas y clases
DATA_ROOT = Path('data/raw')  # raíz donde viven las imágenes crudas

# Clases esperadas (puedes ajustar los nombres si tus carpetas son diferentes)
CLASS_MAP = {
    'Natalie_Portman': 0,
    'Scarlett_Johansson': 1,
}

VALID_EXTS = {'.jpg', '.jpeg', '.png'}

assert DATA_ROOT.exists(), '❌ No se encontró data/raw. Crea la estructura indicada y coloca las imágenes.'
print('📂 Ruta de datos:', DATA_ROOT.resolve())

In [ ]:
# Descubrimiento de imágenes por clase
records = []
for cname, label in CLASS_MAP.items():
    cdir = DATA_ROOT / cname
    if not cdir.exists():
        print(f'⚠️ Carpeta ausente: {cdir} — se omitirá')
        continue
    for p in cdir.rglob('*'):
        if p.suffix.lower() in VALID_EXTS:
            records.append({'path': str(p), 'class': cname, 'label': label})

df = pd.DataFrame(records)
print('Total de imágenes encontradas:', len(df))
display(df.head())
assert len(df) > 0, '❌ No se encontraron imágenes con extensiones válidas (jpg/jpeg/png).'


## 1. Exploración general: conteos, formatos, resoluciones, corruptas, balance


In [ ]:
# Conteo de imágenes por clase
conteo = df['class'].value_counts().rename_axis('clase').reset_index(name='imagenes')
print('📊 Conteo por clase:')
display(conteo)

# Lectura de metadatos de cada imagen
w_list, h_list, fmt_list, corrupt_flags = [], [], [], []
for p in tqdm(df['path'], desc='Leyendo metadatos de imágenes'):
    try:
        with Image.open(p) as im:
            w, h = im.size
            w_list.append(w)
            h_list.append(h)
            fmt_list.append(im.format)
            corrupt_flags.append(False)
    except Exception:
        # Imagen dañada o ilegible
        w_list.append(np.nan)
        h_list.append(np.nan)
        fmt_list.append('CORRUPT')
        corrupt_flags.append(True)

df['width'] = w_list
df['height'] = h_list
df['format'] = fmt_list
df['is_corrupt'] = corrupt_flags

print('🧨 Imágenes corruptas detectadas:', df['is_corrupt'].sum())
print('\n📁 Formatos encontrados:')
display(df['format'].value_counts())

# Histograma de tamaños
plt.figure()
plt.hist(df['width'].dropna(), bins=30)
plt.title('Distribución de anchos de imagen')
plt.xlabel('width (px)'); plt.ylabel('Frecuencia')
plt.show()

plt.figure()
plt.hist(df['height'].dropna(), bins=30)
plt.title('Distribución de altos de imagen')
plt.xlabel('height (px)'); plt.ylabel('Frecuencia')
plt.show()


## 2. Estadística de color (RGB), rango global y detección de imágenes oscuras / borrosas


In [ ]:
def image_stats_rgb(path):
    """Regresa mean RGB, std RGB, valor mínimo y máximo global de la imagen."""
    with Image.open(path) as im:
        im = im.convert('RGB')
        arr = np.array(im)
        mean = arr.mean(axis=(0,1))      # [R,G,B]
        std = arr.std(axis=(0,1))       # [R,G,B]
        mn = arr.min()
        mx = arr.max()
    return mean, std, mn, mx

def variance_of_laplacian(image_bgr):
    """Varianza del Laplaciano: medida de nitidez (baja => borrosa)."""
    return cv2.Laplacian(image_bgr, cv2.CV_64F).var()

rgb_mean, rgb_std, min_vals, max_vals, brightness, blurriness = [], [], [], [], [], []

for p in tqdm(df['path'], desc='Calculando estadísticas RGB / brillo / blur'):
    try:
        m, s, mn, mx = image_stats_rgb(p)
        rgb_mean.append(m)
        rgb_std.append(s)
        min_vals.append(mn)
        max_vals.append(mx)

        # Brillo medio en escala de grises
        with Image.open(p) as im:
            im_gray = im.convert('L')
            brightness.append(float(np.array(im_gray).mean()))

        # Blur usando varianza de Laplaciano
        img_bgr = cv2.imread(p)
        if img_bgr is not None:
            blurriness.append(float(variance_of_laplacian(img_bgr)))
        else:
            blurriness.append(np.nan)
    except Exception:
        rgb_mean.append([np.nan, np.nan, np.nan])
        rgb_std.append([np.nan, np.nan, np.nan])
        min_vals.append(np.nan)
        max_vals.append(np.nan)
        brightness.append(np.nan)
        blurriness.append(np.nan)

# Añadimos columnas al dataframe
df[['mean_R', 'mean_G', 'mean_B']] = pd.DataFrame(rgb_mean, index=df.index)
df[['std_R', 'std_G', 'std_B']] = pd.DataFrame(rgb_std, index=df.index)
df['min_val'] = min_vals
df['max_val'] = max_vals
df['brightness'] = brightness
df['blurriness'] = blurriness

print('🎯 Estadísticas promedio por clase (RGB):')
display(df[['class','mean_R','mean_G','mean_B','std_R','std_G','std_B']].groupby('class').mean())

print('🌈 Rango global de valores de pixel [min, max]:', int(df['min_val'].min()), int(df['max_val'].max()))

# Definimos umbrales simples para marcar imágenes oscuras y borrosas
DARK_THR = 35   # brillo medio bajo => muy oscura
BLUR_THR = 70   # varianza de Laplaciano baja => borrosa (ajustable)

df['flag_dark'] = df['brightness'] < DARK_THR
df['flag_blur'] = df['blurriness'] < BLUR_THR

print('\n📌 Proporción de imágenes oscuras / borrosas por clase:')
display(df.groupby('class')[['flag_dark','flag_blur']].mean())
print('\nTotal marcadas como oscuras:', int(df['flag_dark'].sum()))
print('Total marcadas como borrosas:', int(df['flag_blur'].sum()))

## 3. Detección de duplicados por *perceptual hash* (pHash)


In [ ]:
hashes = []
for p in tqdm(df['path'], desc='Calculando perceptual hash'):
    try:
        with Image.open(p) as im:
            h = imagehash.phash(im.convert('RGB'))
            hashes.append(str(h))
    except Exception:
        hashes.append('CORRUPT')

df['phash'] = hashes

# Agrupamos por hash para ver posibles duplicados
dup_groups = df.groupby('phash').filter(lambda g: len(g) > 1).sort_values('phash')
print('🔁 Posibles duplicados detectados:', dup_groups.shape[0])
display(dup_groups.head(10))

## 4. Preprocesamiento y Aumento de Datos (Torchvision)

Definimos dos pipelines de transformaciones:
- `basic_tf`: para validación/inferencia (solo resize + normalización).
- `aug_tf`: para entrenamiento (incluye cambios de color, flips, rotaciones, recortes, etc.).


In [ ]:
IMG_SIZE = (224, 224)  # tamaño clásico para modelos tipo ResNet, etc.

basic_tf = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

aug_tf = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.05),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=12),
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.85, 1.0), ratio=(0.95, 1.05)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

print('✅ Transforms definidos (básico y con data augmentation)')

## 5. División estratificada train/val sin sklearn

Hacemos un split 80/20 manteniendo el balance entre clases usando solo NumPy y Pandas.


In [ ]:
def stratified_split(df, label_col='label', test_size=0.2, seed=42):
    """División estratificada simple usando NumPy (sin sklearn)."""
    rng = np.random.default_rng(seed)
    idx_train, idx_val = [], []
    for lbl, g in df.groupby(label_col):
        ids = g.index.to_numpy()
        rng.shuffle(ids)
        n_val = int(len(ids) * test_size)
        idx_val.extend(ids[:n_val])
        idx_train.extend(ids[n_val:])
    train_df = df.loc[idx_train].reset_index(drop=True)
    val_df = df.loc[idx_val].reset_index(drop=True)
    return train_df, val_df

train_df, val_df = stratified_split(df, label_col='label', test_size=0.2, seed=42)

print('Tamaño TRAIN:', len(train_df))
print('Tamaño VAL  :', len(val_df))

print('\nDistribución por clase en TRAIN:')
display(train_df['class'].value_counts())
print('\nDistribución por clase en VAL:')
display(val_df['class'].value_counts())

## 6. Exportación de CSVs para control de calidad (QC)

Se generan varios archivos en `data/qc/`:
- `inventory_full.csv`: inventario completo con metadatos.
- `train_list.csv` / `val_list.csv`: listas para cada split.
- `to_review_flags.csv`: imágenes oscuras, borrosas o corruptas.
- `possible_duplicates.csv`: candidatos duplicados según pHash.


In [ ]:
qc_path = Path('data/qc')
qc_path.mkdir(parents=True, exist_ok=True)

df.to_csv(qc_path / 'inventory_full.csv', index=False)
train_df.to_csv(qc_path / 'train_list.csv', index=False)
val_df.to_csv(qc_path / 'val_list.csv', index=False)

flags_df = df[df['is_corrupt'] | df['flag_dark'] | df['flag_blur']]
flags_df.to_csv(qc_path / 'to_review_flags.csv', index=False)

dup_groups.to_csv(qc_path / 'possible_duplicates.csv', index=False)

print('💾 Archivos de control de calidad guardados en:', qc_path.resolve())

## 7. Dataset y DataLoaders de PyTorch (opcional para entrenamiento)

A partir de `train_df` y `val_df` definimos un `Dataset` y `DataLoader` estándar, listo para entrenar un modelo (por ejemplo, una `ResNet18`).


In [ ]:
class FaceDataset(torch.utils.data.Dataset):
    def __init__(self, df, transform):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(row['path']).convert('RGB')
        x = self.transform(img)
        y = int(row['label'])
        return x, y

train_ds = FaceDataset(train_df, aug_tf)
val_ds = FaceDataset(val_df, basic_tf)

train_loader = torch.utils.data.DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=2, pin_memory=True)
val_loader = torch.utils.data.DataLoader(val_ds, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)

print('✅ DataLoaders listos. Puedes usar train_loader y val_loader para entrenar tu modelo.')

## 8. Checklist final de la tarea

- [x] No se utilizó `scikit-learn`.
- [x] Explorado el dataset: conteo, formatos, resoluciones, imágenes corruptas.
- [x] Análisis de color por canal RGB y rangos globales.
- [x] Detección de imágenes oscuras y borrosas.
- [x] Detección de posibles duplicados (pHash).
- [x] Definidas transformaciones básicas y de aumento de datos.
- [x] División estratificada train/val (80/20) hecha manualmente.
- [x] Exportados CSVs de control de calidad en `data/qc/`.
- [x] Definidos `Dataset` y `DataLoader` de PyTorch listos para entrenamiento.
